# Day 12: Garden Groups

In [ ]:
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt

with open('data/2024-12-example.txt', 'r') as f:
    test_data = f.readlines()
with open('data/2024-12.txt', 'r') as f:
    data = f.readlines()

test_data = [
    'EEEEE',
    'EXXXX',
    'EEEEE',
    'EXXXX',
    'EEEEE'
]


data = [x.strip() for x in data]
test_data = [x.strip() for x in test_data]

In [ ]:
def label_regions(str_data: list, plot: bool = False):
    # convert input data to array    
    plant_types = list(set(''.join(str_data)))
    arr = np.zeros((len(str_data), len(str_data[0])))
    for i in range(len(str_data)):
        for j in range(len(str_data[i].strip())):
            arr[i, j] = plant_types.index(str_data[i][j]) + 1
    
    if plot:
        plt.imshow(arr)
        plt.axis('off')
        plt.title('Areas')
        plt.show()

    return arr


def label_areas(region_array, plot: bool = False):
    # label the regions
    label_array = measure.label(region_array.astype(int), connectivity=1)

    if plot:
        plt.imshow(label_array)
        plt.title('Labeled Areas')
        plt.axis('off')
        plt.show()
    return label_array


def calculate_perimeter(label_array, plot: bool = False):
    # start with all perimeters as 4
    perimeters = np.ones_like(label_array) * 4
    
    # remove count for each adjacent pixel with the same label
    for i in range(label_array.shape[0]):
        for j in range(label_array.shape[1]):
            c_label = label_array[i, j]
            # left side
            if i > 0:
                if c_label == label_array[i-1, j]:
                    perimeters[i, j] -= 1
            # right side
            if i < label_array.shape[0] - 1:
                if c_label == label_array[i+1, j]:
                    perimeters[i, j] -= 1
            # top side
            if j > 0:
                if c_label == label_array[i, j-1]:
                    perimeters[i, j] -= 1
            # bottom side
            if j < label_array.shape[1] - 1:
                if c_label == label_array[i, j+1]:
                    perimeters[i, j] -= 1

    
    if plot:
        plt.imshow(perimeters)
        plt.title('Perimeters')
        plt.axis('off')
        plt.show()
    
    return perimeters


def compute_price_perimeter(label_array, perimeter):
    total_price = 0
    for region in measure.regionprops(label_array):
        # compute perimeter of region
        region_perimeter = 0
        for c in region.coords:
            region_perimeter += perimeter[c[0], c[1]]

        # compute cost of region
        total_price += region_perimeter * int(region.area)
    return total_price

## Part 1

In [ ]:
region_array = label_regions(test_data, plot=False)
labels = label_areas(region_array, plot=False)
perimeters = calculate_perimeter(labels, plot=False)
print('Part 1 (test):', compute_price_perimeter(labels, perimeters))

In [ ]:
# region_array = label_regions(data, plot=False)
# labels = label_areas(region_array, plot=False)
# perimeters = calculate_perimeter(labels, plot=False)
# print('Part 1:', compute_price_perimeter(labels, perimeters))

## Part 2

In [ ]:
def calculate_price_sides(label_array, perimeter):
    total_price = 0
    for region in measure.regionprops(label_array):
        # compute number of sides in region
        region_sides = 0
        for c in region.coords:
            region_perimeter += perimeter[c[0], c[1]]

        # compute cost of region
        total_price += region_perimeter * int(region.area)
    return total_price


In [ ]:
for region in measure.regionprops(labels):
    reg_coords = region.coords
    
    # split into X and Y groups to find connected regions
    sorted_x = reg_coords[reg_coords[:, 0].argsort()]
    sorted_y = reg_coords[reg_coords[:, 1].argsort()]
    groups_x = np.split(sorted_x[:, 1], np.unique(sorted_x[:, 0], return_index=True)[1][1:])
    groups_y = np.split(sorted_y[:, 0], np.unique(sorted_y[:, 1], return_index=True)[1][1:])
    
    
    for i in range(len(groups_x)-1):
        break